In [1]:
import imaplib
import pandas as pd
import email
from bs4 import BeautifulSoup



mail = imaplib.IMAP4_SSL('imap.gmail.com', 993)
mail.login('lodge.brennan@gmail.com', 'PASSWORD')
#mail.select('"[Gmail]/tub06054@temple.edu"')
mail.select('"spam_research"')
result, data = mail.search(None, '(SINCE "01-Jan-2012")')
uids = data[0].split()





In [2]:
#Loop through each uid to fetch headers
import sqlite3 as db
import re
msg_dict = None
newdata = pd.DataFrame()
for uid in uids:
    if result == 'OK':
    
        resp, data = mail.fetch(uid, "(RFC822)")
        
        full_message = ""
        msg_dict = None
        df = pd.DataFrame()
        
        body_base64 = ""

        raw_email = data[0][1] # here's the body, which is raw text of the whole email

        # including headers and alternate payloads
              
        #msg = email.message_from_bytes(raw_email)
        
        msg = email.message_from_string(str(raw_email, 'utf-8'))
        #print(msg)

        msg_items = msg.items()
        msg_dict = dict(msg_items)

                
        body_part = {}

        for part in msg.walk():
            body_part = {
                #"Message-ID": key,
                "MESSAGE_TYPE": str(part.get_content_maintype()),
                "CHARSETS": str(part.get_charsets()),
                "DEFULAT_TYPE:": str(part.get_default_type()),
                "CONTENT_CHARSET:": str(part.get_content_charset())}

        body = {}
        text = ""
        '''
        check = ""
        parser = email.parser.Parser()
        parts = msg.get_payload()
        check = parts[0].get_content_type()
        if msg.is_multipart():
            if check == "text/plain":
                text = parts[0].get_payload()

            #print parts[0].get_payload()
            elif check == "multipart/alternative":
                part = parts[0].get_payload()

            #print parts[0].get_payload()
                if part[0].get_content_type() == "text/plain":
                    try:
                        text = part[0].get_payload()
                    except:
                        pass
        '''
        if msg.is_multipart():
            for part in msg.walk():

                try:
                    text = part.get_payload(decode=True)
                    missing_padding = 4 - len(text) % 4

                    if missing_padding:
                        text += b'='* missing_padding

                        text += base64.urlsafe_b64decode(text)
                        #print body_base64 + "&"*80
                    else:
                        text += base64.urlsafe_b64decode(body)
                        #print body_base64 + "&"*80

                except:
                    pass
            else:
                if text is None:
                        charset = str(msg.get_content_charset())
                        text = unicode(msg.get_payload(decode=True),charset, "ignore").encode('utf8','ignore')
                        text = text.strip()
            '''
            #if text is not None:
                #text = text.strip()
            #else:
                #html = html.strip()

            '''
        #text = BeautifulSoup(text.decode('utf-16', 'ignore'))
        #remove uncessary spaces
        def removeinvalid(y):
            y=''.join([x for x in y if ord(x) < 128])
            return y
        soup = BeautifulSoup(text,features='html.parser')
        text = re.sub(r'[\n\r\t]{2,}', ' ', soup.get_text(separator=u"\n"))
        text = removeinvalid(str(text))
        text = text.encode('utf-8')
        msg_dict['body_part'] = body_part
        msg_dict['body'] = text
        msg_dict = {k:[v] for k,v in msg_dict.items()}
        
        #append new msg dict rows to dataframe 
        df_new = pd.DataFrame([msg_dict])
        df_new.columns = df_new.columns.str.lower()
        #drop duplicate columns 
        df_new = df_new.loc[:, ~df_new.columns.duplicated()]
        newdata = pd.concat([newdata, df_new])    
        #convert to dataframe and ensure all values are strings for sqlite
        newdata = pd.DataFrame(newdata)
        newdata_spam = newdata.applymap(str)
        
        
    else:
        pass


In [3]:
newdata_spam

,delivered-to,received,x-google-smtp-source,x-received,return-path,received-spf,authentication-results,arc-seal,arc-message-signature,arc-authentication-results,...,x-job,x-orgid,x-emarsys-identify,x-emarsys-environment,x-kmail-relay,x-feedback-id,x-sg-eid,x-sg-id,x-entity-id,x-qumid
0,['lodge.brennan@gmail.com'],['from michaelivanchenko.com (13.82.233.122) b...,['AA6agR4mSeoFwzk57ey2LRfANTNY5pEO2G6gPHU5VJao...,['by 2002:a17:907:628a:b0:72f:678d:6047 with S...,['<>'],['None (protection.outlook.com: michaelivanche...,['mx.google.com;\r\n arc=fail (signature...,['i=1; a=rsa-sha256; s=arcselector9901; d=micr...,['i=1; a=rsa-sha256; c=relaxed/relaxed; d=micr...,['i=1; mx.microsoft.com 1; spf=none (sender ip...,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0,['lodge.brennan@gmail.com'],['from localhost (localhost [127.0.0.1])\r\n\t...,['AA6agR4zJJIKFRP8wQ3RXinjgVEMcMJyIyYLt1hI0e6l...,['by 2002:a25:2543:0:b0:67f:d33d:53f1 with SMT...,['<bounce-mc.us1_155336962.9430097-96ca729718@...,['pass (google.com: domain of bounce-mc.us1_15...,['mx.google.com;\r\n dkim=pass header.i=...,['i=1; a=rsa-sha256; t=1660302879; cv=none;\r\...,['i=1; a=rsa-sha256; c=relaxed/relaxed; d=goog...,['i=1; mx.google.com;\r\n dkim=pass head...,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0,['lodge.brennan@gmail.com'],['from mta-70-52-143.shared.klaviyomail.com (m...,['AA6agR5EbC9UgfS6k0tHoP+jmNJ7YixaXrtJ8rP0YTm7...,['by 2002:a17:90b:388f:b0:1f5:7748:9667 with S...,['<msprvs1=19223iaGrzP9N=bounces-304613-38945@...,['pass (google.com: domain of msprvs1=19223iag...,['mx.google.com;\r\n dkim=pass header.i=...,['i=1; a=rsa-sha256; t=1660296720; cv=none;\r\...,['i=1; a=rsa-sha256; c=relaxed/relaxed; d=goog...,['i=1; mx.google.com;\r\n dkim=pass head...,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0,['lodge.brennan@gmail.com'],['from tradefromeurope.com (20.7.142.115) by\r...,['AA6agR7Vkwe42+M5EqLdRzoejAMi5sZALIigFA9lOxrF...,['by 2002:aa7:cb83:0:b0:443:3f15:84fe with SMT...,['<>'],['None (protection.outlook.com: tradefromeurop...,['mx.google.com;\r\n arc=fail (signature...,['i=1; a=rsa-sha256; s=arcselector9901; d=micr...,['i=1; a=rsa-sha256; c=relaxed/relaxed; d=micr...,['i=1; mx.microsoft.com 1; spf=none (sender ip...,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0,['snort-users@lists.snort.org'],['from YQXPR01MB3990.CANPRD01.PROD.OUTLOOK.COM...,['AA6agR47rMK50lQLItSDetGeGHimob020axtGVgo8iy7...,['by 2002:a63:90c4:0:b0:41c:ccc9:16b2 with SMT...,['<snort-users-bounces@lists.snort.org>'],['pass (google.com: domain of snort-users-boun...,"['sfo.mailmanlists.us;\r\n\tdkim=fail reason=""...",['i=1; a=rsa-sha256; s=arcselector9901; d=micr...,['i=1; a=rsa-sha256; c=relaxed/relaxed; d=micr...,['i=1; mx.microsoft.com 1; spf=pass\r\n smtp.m...,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0,['lodge.brennan@gmail.com'],['from go-ixesoft.com (20.127.19.70) by\r\n MW...,['AA6agR6W+vlHdCwwaTqICrWmkvngmNxndfh7zcJk0kj2...,['by 2002:a05:6402:50cc:b0:43e:6860:58fc with ...,['<>'],['None (protection.outlook.com: go-ixesoft.com...,['mx.google.com;\r\n arc=fail (signature...,['i=1; a=rsa-sha256; s=arcselector9901; d=micr...,['i=1; a=rsa-sha256; c=relaxed/relaxed; d=micr...,['i=1; mx.microsoft.com 1; spf=none (sender ip...,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0,['lodge.brennan@gmail.com'],['from go-ixesoft.com (20.127.19.70) by\r\n BN...,['AA6agR7MmOLaRFn+JXs7fqOEC6fLFAggVw0FSofDvqVb...,['by 2002:a17:907:d0f:b0:731:5c2:a9a5 with SMT...,['<>'],['None (protection.outlook.com: go-ixesoft.com...,['mx.google.com;\r\n arc=pass (i=1);\r\n...,['i=1; a=rsa-sha256; s=arcselector9901; d=micr...,['i=1; a=rsa-sha256; c=relaxed/relaxed; d=micr...,['i=1; mx.microsoft.com 1; spf=none (sender ip...,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0,['lodge.brennan@gmail.com'],['from dabtechfoods.com (20.12.66.127) by\r\n ...,['AA6agR4W2z32O1Ik8cLC55WSvbyWO2ZHBczeQiqL+csy...,['by 2002:a17:906:9b83:b0:730:b3ae:343 with SM...,['<>'],['None (protection.outlook.com: dabtechfoods.c...,['mx.google.com;\r\n arc=pass (